<a href="https://colab.research.google.com/github/Centrattic/SEESZooniverse/blob/main/data_processing/downloading_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import shutil
import time
import random
from google.colab import drive
import pandas as pd
from tqdm import tqdm
import os
from PIL import Image
import cv2
import numpy as np

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(os.listdir("drive/MyDrive/Zooniverse_Images2"))

5896

In [ ]:
!mkdir "drive/MyDrive/Zooniverse_Images2/"

In [ ]:
def download_image(image_url, image_filename, folder_path = 'drive/MyDrive/Zooniverse_Images2/', image_size = (512, 512)):
  # Download the image
  target_path = folder_path + image_filename
  if os.path.exists(target_path):
    print('Already there.')
  else:
      response = requests.get(image_url, stream=True)
      if response.status_code == 200:
      # Create a temporary file to store the downloaded image
        image_file = '/content/temp_image.jpg'
        with open(image_file, 'wb') as file:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, file)

        img = Image.open("temp_image.jpg")
        img = img.resize(image_size)

        img.save(target_path)

      # Print the path of the downloaded image in Google Drive
      #print('Image downloaded successfully. File saved at:', target_path)
  # else:
      #print('Failed to download the image.')

In [ ]:
data = pd.read_csv("drive/MyDrive/Data_Files/SEES2023.csv")
data = data.drop(data.columns[0], axis=1) # dropping random column that came in

In [ ]:
data.head()

,protocol,measuredDate,createDate,updateDate,publishDate,organizationId,organizationName,siteId,siteName,countryName,...,landcoversRainingSnowing,landcoversUpwardExtraData,landcoversMeasurementElevation,landcoversWestExtraData,landcoversLandCoverId,landcoversMeasurementLongitude,landcoversMucDescription,landcoversSnowIce,landcoversNorthClassifications,landcoversFieldNotes
0,land_covers,2023-07-13,2023-07-13T14:45:10,2023-07-13T15:15:01,2023-07-14T04:11:09,17043304.0,United States of America Citizen Science,320976.0,18TWK209668,United States,...,False,NaN,56.6,NaN,60993.0,-74.7529,"Herbaceous/Grassland, Short Grass",False,"30% MUC 02 (b) [Trees, Closely Spaced, Deciduo...",Area is in my backyard and in a shaded and woo...
1,land_covers,2023-07-13,2023-07-13T14:45:10,2023-07-13T15:15:01,2023-07-14T04:11:09,17043304.0,United States of America Citizen Science,320976.0,18TWK209668,United States,...,False,NaN,56.8,NaN,60994.0,-74.7529,"Herbaceous/Grassland, Short Grass",False,"20% MUC 01 (b) [Trees, Closely Spaced, Evergre...",Area is in my backyard and between two houses....
2,land_covers,2023-07-11,2023-07-12T16:40:07,2023-07-18T03:04:04,2023-07-18T03:10:00,17043304.0,United States of America Citizen Science,321510.0,18TXL010372,United States,...,False,NaN,81.0,NaN,60960.0,-73.7994,NaN,False,NaN,(none)
3,land_covers,2023-07-11,2023-07-12T16:40:24,2023-07-18T03:02:22,2023-07-18T03:10:00,17043304.0,United States of America Citizen Science,321523.0,18TXL020368,United States,...,False,NaN,79.8,NaN,60973.0,-73.7871,NaN,False,NaN,(none)
4,land_covers,2023-07-11,2023-07-12T16:40:08,2023-07-18T03:01:07,2023-07-18T03:10:00,17043304.0,United States of America Citizen Science,321509.0,18TXL010377,United States,...,False,NaN,60.3,NaN,60959.0,-73.7992,NaN,False,NaN,(none)


In [ ]:
img_cols = ['landcoversEastPhotoUrl','landcoversNorthPhotoUrl', 'landcoversSouthPhotoUrl', 'landcoversWestPhotoUrl']
new_name_cols = ['landcoversEastPhotoName','landcoversNorthPhotoName', 'landcoversSouthPhotoName', 'landcoversWestPhotoName']

In [ ]:
data = data.dropna(subset=img_cols).reset_index(drop=True) # dropping if nan in our particular columns, only like three locations
len(data)

1474

In [ ]:
east_names = ['']*len(data)
west_names = ['']*len(data)
south_names = ['']*len(data)
north_names = ['']*len(data)

for i in tqdm(range(len(data))): # of data points
  for name in img_cols:
    image_url = str(data.loc[i, name])
    image_filename = image_url.split('/')[-2] + ".jpg"

    if 'East' in name:
      east_names[i] = image_filename
    elif 'West' in name:
      west_names[i] = image_filename
    elif 'South' in name:
      south_names[i] = image_filename
    elif 'North' in name:
      north_names[i] = image_filename

    download_image(image_url, image_filename)

100%|██████████| 1474/1474 [25:55<00:00,  1.06s/it]


In [ ]:
for name in new_name_cols:
    if 'East' in name:
      data[name] = east_names
    elif 'West' in name:
      data[name] = west_names
    elif 'South' in name:
      data[name] = south_names
    elif 'North' in name:
      data[name] = north_names

In [ ]:
1474*4

In [ ]:
len(os.listdir("drive/MyDrive/Zooniverse_Images")) # YEEEESSSS

In [ ]:
data.to_csv("Zooniverse_data.csv")

In [ ]:
zoo = pd.read_csv("drive/MyDrive/Data_Files/Zooniverse_data.csv")

In [ ]:
new_name_cols = ['landcoversEastPhotoName','landcoversNorthPhotoName', 'landcoversSouthPhotoName', 'landcoversWestPhotoName']

In [ ]:
google_earth = ['3446922.jpg', '3446887.jpg']

In [ ]:
zoo[zoo['landcoversUserid'] == 107115175.0]['landcoversWestClassifications']

438     NaN
451     NaN
453     NaN
454     NaN
466     NaN
480     NaN
490     NaN
499     NaN
500     NaN
501     NaN
505     NaN
506     NaN
507     NaN
517     NaN
520     NaN
531     NaN
536     NaN
537     NaN
539     NaN
552     NaN
553     NaN
554     NaN
556     NaN
566     NaN
570     NaN
582     NaN
587     NaN
595     NaN
599     NaN
602     NaN
610     NaN
625     NaN
629     NaN
636     NaN
639     NaN
1385    NaN
1386    NaN
1412    NaN
1423    NaN
1425    NaN
Name: landcoversWestClassifications, dtype: object

In [ ]:
zoo.iloc[453]['landcoversUserid']

107115175.0

In [ ]:
zoo.iloc[556]

318852.0

In [ ]:
zoo.columns

Index(['Unnamed: 0', 'protocol', 'measuredDate', 'createDate', 'updateDate',
       'publishDate', 'organizationId', 'organizationName', 'siteId',
       'siteName', 'countryName', 'countryCode', 'latitude', 'longitude',
       'elevation', 'pid', 'landcoversDownwardPhotoUrl',
       'landcoversEastExtraData', 'landcoversEastPhotoUrl',
       'landcoversLocationAccuracyM', 'landcoversMucCode',
       'landcoversUpwardPhotoUrl', 'landcoversEastCaption',
       'landcoversMeasurementLatitude', 'landcoversWestClassifications',
       'landcoversNorthCaption', 'landcoversNorthExtraData',
       'landcoversDataSource', 'landcoversDryGround', 'landcoversWestCaption',
       'landcoversSouthClassifications', 'landcoversNorthPhotoUrl',
       'landcoversUpwardCaption', 'landcoversDownwardExtraData',
       'landcoversEastClassifications', 'landcoversMucDetails',
       'landcoversLocationMethod', 'landcoversMeasuredAt',
       'landcoversDownwardCaption', 'landcoversSouthPhotoUrl',
       'lan